In [2]:
def get_u_d(S_0, S_u, S_d):
    ''' Get by how much the up and down nodes for
        the stock price increase/decrease.
    '''
    return (S_u/S_0, S_d/S_0)

def get_qs(u,d,r):
    ''' Get probabilities of the up and down movements.
    '''
    q_u = (1+r-d)/(u-d)
    q_d = 1-q_u
    return (q_u,q_d)

def get_qs(u,d,r):
    ''' Get probabilities of the up and down movements.
    '''
    q_u = (1+r-d)/(u-d)
    q_d = 1-q_u
    return (q_u,q_d)

def get_price_at_merge(r, q_u, C_u, C_d):
    ''' Get cashflow for one year maturity
    '''
    q_d = 1-q_u
    return (q_u*C_u+ q_d*C_d)/(1+r)

def get_price_at_merge_two(r, q_u, C_u, C_d):
    ''' Get cashflow for two year maturity
        Inputs:
        end_cashflows(array of tuples) [(C_uu,C_ud),(C_du,C_dd)]
        replaces (q_u**2*C_uu+q_u*q_d*C_ud+q_d*q_u*C_du+q_d**2*C_dd)/(1+r)**2
    '''
    q_d = 1-q_u
    return (q_u*C_u+ q_d*C_d)/(1+r)

'''
What if we branch like this [[],[],[],[]].. and length of payoffs determines
the branching factor
'''

def price_option(r,S_0,stock_prices,payoffs):
    S_u, S_d= stock_prices
    u,d = get_u_d(S_0,S_u,S_d)
    q_u, _ = get_qs(u,d,r)
    C_u = get_price_at_merge(r, q_u, payoffs[0][0],payoffs[0][1])
    C_d = get_price_at_merge(r, q_u, payoffs[1][0],payoffs[1][1])
    C_0 = get_price_at_merge(r, q_u, C_u, C_d)
    return C_0


In [3]:
def test_q1():
    assert (
        round( price_option(0.05,100,(125,80),[(56.25,0),(20,0)]), 4) ==
        20.2262)
    assert (
        round(price_option(0.05,300,(420,210),[(1,0),(0,0)]),4) ==
        0.2268)
        
test_q1()

In [4]:
import math

def get_share_amount(s_0, c_u, c_d, u, d):
    return (c_u-c_d)/((u-d)*s_0)

def get_bond_amount(r, c_u, c_d, u, d):
    return (1/(1+r))*((u*c_d-d*c_u)/(u-d))

def construct_portfolios(r, stock_prices, payoffs):
    
    s_0 = stock_prices[0]
    s_1 = stock_prices[1]
    u,d = get_u_d(s_0[0],s_1[0],s_1[1])
    q_u, _ = get_qs(u,d,r)

    portfolio = []
    price_index = -1
    time_interval = math.log(len(payoffs),2)

    while len(payoffs)//2 > 0:

        new_prices = []
        change_index = 0

        for i in range(0,len(payoffs),2):
            c_u, c_d = payoffs[i], payoffs[i+1]
            s_t = stock_prices[price_index][change_index]
            delta = get_share_amount(s_t, c_u, c_d, u, d)
            b = get_bond_amount(r, c_u, c_d, u, d)
            new_prices.append(get_price_at_merge(r, q_u, c_u, c_d))
            movement = "up" if change_index%2 == 0 else "down"
            portfolio.append({"time":time_interval+price_index,
                                "movement":movement,
                                "share_amount":delta,
                                "bound_amount":b})
            change_index += 1

        payoffs = new_prices
        print(payoffs)
        price_index -= 1

    return portfolio

construct_portfolios(0.05,[[100],[125,80]],[56.25,0,20,0])


[29.76190476190476, 10.58201058201058]
[20.22619747487472]


[{'time': 1.0,
  'movement': 'up',
  'share_amount': 1.0000000000000002,
  'bound_amount': -95.23809523809524},
 {'time': 1.0,
  'movement': 'down',
  'share_amount': 0.5555555555555556,
  'bound_amount': -33.86243386243386},
 {'time': 0.0,
  'movement': 'up',
  'share_amount': 0.4262198706643151,
  'bound_amount': -22.395789591556795}]

In [16]:
def get_u_d(S_0, S_u, S_d):
    ''' Get by how much the up and down nodes for
        the stock price increase/decrease.
    '''
    return (S_u/S_0, S_d/S_0)

def get_q(u,d,r):
    ''' Get probabilities of the up and down movements.
    '''
    return (1+r-d)/(u-d)

def get_price_at_merge(r, q_u, C_u, C_d):
    ''' Get cashflow for one year maturity
    '''
    q_d = 1-q_u
    return (q_u*C_u+q_d*C_d)/(1+r)


s0, su, sd = 298, 444.02, 268.2
cu,cd =244.02, 68.02
r = 0.03

u, d = get_u_d(S_0=s0, S_u=su, S_d=sd)
qu, _ = get_qs(u=u,d=d,r=r)

c0 = get_price_at_merge(r, q_u=qu, C_u=cu, C_d=cd)
c0

103.68899127859143

In [11]:
s0, su, sd = 200, 298, 180
cu,cd=104.2580, 14.5894
r = 0.03

u, d = get_u_d(S_0=s0, S_u=su, S_d=sd)
qu, qd = get_qs(u=u,d=d,r=r)

delta = get_share_amount(s0, c_u=cu, c_d=cd, u=u, d=d)
b = get_bond_amount(r, c_u=cu, c_d=cd, u=u, d=d)
delta

0.7599033898305084